In [1]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, chi2

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from bs4 import BeautifulSoup
from langdetect import detect
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

C:\Users\sdidd\AppData\Local\Temp\ipykernel_24388\1083772421.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Pre Processing

In [2]:
df = pd.read_csv("../Dataset/IMDB Dataset.csv")
df.head()
df = df.rename(columns={'review': 'OriginalReviews'})
df = df.rename(columns={'sentiment': 'OutputSentiment'})
df_subset = df.sample(n=5000).reset_index(drop=True)
df_subset.head()
df_subset['OutputSentiment'].value_counts()

OutputSentiment
negative    2532
positive    2468
Name: count, dtype: int64

In [3]:
# Function to remove HTML tags from a text
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

#Stopwords removal
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from a text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Function to remove URLs from a text
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)

# Function to filter non-English comments
def filter_non_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:

# Remove numbers from the 'reviews' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('\d+', '')

#Remove punctuations
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('[{}]'.format(string.punctuation), '')

# Apply the remove_stopwords function to the 'reviews' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_stopwords)

# Apply the remove_html_tags function to the 'text' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_html_tags)

# Apply the remove_urls function to the 'reviews' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_urls)

# Create a boolean mask for non-English reviews
mask = df_subset['OriginalReviews'].apply(filter_non_english)

# Create a new DataFrame containing only English reviews
df_subset = df_subset[mask]

nltk.download('wordnet')
# Lemmatization function
def lemmatize_column(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized_words)

# Apply the function to the specific column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(lemmatize_column)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sdidd\AppData\Local\Temp\ipykernel_24388\3669464969.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('\d+', '')
C:\Users\sdidd\AppData\Local\Temp\ipykernel_24388\3685019402.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df_subset.to_csv("../csv/Preprocessed_data.csv",index=False)

## Feature Extraction Using TF-IDF

In [6]:
preprocessed = pd.read_csv('../csv/Preprocessed_data.csv')

In [7]:
num_features_to_keep = 13000

# Create a pipeline with TfidfVectorizer and SelectKBest
pipeline = make_pipeline(TfidfVectorizer(), SelectKBest(f_classif, k=num_features_to_keep))

# Fit and transform your data
X_transformed = pipeline.fit_transform(preprocessed['OriginalReviews'], preprocessed['OutputSentiment'])

# Get the selected feature names
selected_feature_names = pipeline.named_steps['tfidfvectorizer'].get_feature_names_out()[pipeline.named_steps['selectkbest'].get_support()]

# Create a DataFrame with the selected features
selected_features_df = pd.DataFrame(X_transformed.toarray(), columns=selected_feature_names)

# Concatenate the existing DataFrame with the new selected features DataFrame
tfidf_df_13k = pd.concat([preprocessed, selected_features_df], axis=1)

tfidf_df_13k.head()

tfidf_df_13k.to_csv("../csv/tfidf_df_13k.csv")

## CONNOTATIONS

In [8]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
delimiter = '\t'

# Read the text file into a DataFrame
positive = pd.read_csv(r'..\Connotations\positive-words.txt', sep=delimiter, names=['words'])
negative = pd.read_csv(r'..\Connotations\negative-words.txt', sep=delimiter, names=['words'])
connotations = pd.read_csv(r"..\Connotations\connotations.csv")

word_emotion_map = dict(zip(connotations['word'], connotations['emotion']))

def update_counts(review):
    positive_count = sum(1 for word in review.split() if word in word_emotion_map and word_emotion_map[word] == 'positive')
    negative_count = sum(1 for word in review.split() if word in word_emotion_map and word_emotion_map[word] == 'negative')
    return positive_count, negative_count

tfidf_df_13k[['Positive_Connotation_Count', 'Negative_Connotation_Count']] = tfidf_df_13k['OriginalReviews'].apply(update_counts).tolist()

In [10]:
# Load positive and negative words from files
positive_words_df = pd.read_csv(r'..\Connotations\positive-words.txt', header=None, names=['words'])
negative_words_df = pd.read_csv(r'..\Connotations\negative-words.txt', header=None, names=['words'])

# Convert DataFrame columns to sets
positive_words = set(positive_words_df['words'].tolist())
negative_words = set(negative_words_df['words'].tolist())

# Assuming 'tfidf_df_13k' is your DataFrame
# Define a function to update counts based on positive and negative words
def update_word_counts(review):
    positive_count = sum(1 for word in review.split() if word in positive_words)
    negative_count = sum(1 for word in review.split() if word in negative_words)
    return positive_count, negative_count

# Apply the function to the 'OriginalReviews' column and unpack the result into two new columns
tfidf_df_13k[['Positive_Word_Count', 'Negative_Word_Count']] = tfidf_df_13k['OriginalReviews'].apply(update_word_counts).tolist()
tfidf_df_13k.to_csv("../csv/tfidf_df_13k_connotations.csv")

In [11]:
tfidf_df_13k

,OriginalReviews,OutputSentiment,007,0083,01,03,04,10,1000,101,...,ànd,ça,élan,émigré,émigrés,était,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count
0,would one funniest TV sitcom come States since...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,35,18,10,4
1,opportunity preview film member test audience ...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20,22,2,5
2,even willing vote single star crap IMDb does't...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,28,26,1,10
3,lost count many review 've written Slaughter H...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,54,35,13,13
4,"Seven Ups compared Bullitt chase scene , come ...",positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40,46,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,Franco Rossi 's 1985 six-hour Italian mini-ser...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,66,50,11,8
4995,"Hands , best drama/comedy show television . cl...",positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,26,23,9,2
4996,"9 , film 've looking forward month .... little...",negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31,28,9,9
4997,Prince Central Park ( 2000 ) utterly bad . pur...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,6,1,3


In [12]:
tfidf_df_13k = pd.read_csv("../csv/tfidf_df_13k.csv")

In [13]:
tfidf_df_13k_connotations = pd.read_csv('../csv/tfidf_df_13k_connotations.csv')

In [14]:
tfidf_df_13k_connotations = tfidf_df_13k_connotations.drop('Unnamed: 0',axis=1)

In [15]:
df_statistical = tfidf_df_13k_connotations.drop(columns=['OriginalReviews','Positive_Connotation_Count','Negative_Connotation_Count','Positive_Word_Count','Negative_Word_Count'], axis=1)
df_statistical.head()

,OutputSentiment,007,0083,01,03,04,10,1000,101,102,...,zulu,zwick,zz,zázvorková,ànd,ça,élan,émigré,émigrés,était
0,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
label = LabelEncoder()
df_statistical['OutputSentiment'] = label.fit_transform(df_statistical['OutputSentiment'])

## CHI SQAURE

In [17]:
# This will get the top 5000 relavant features out of the sample
chi2_selector = SelectKBest(chi2, k=5000)

# This will transform the dataset i.e, it will reduce the dimensions by just considering the relavant features only
X = df_statistical.drop(columns=['OutputSentiment'])
y = df_statistical['OutputSentiment']
X_5000 = chi2_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X.columns[selected_feature_indices]

chisq_5k = X[selected_feature_names]
chisq_5k.head()

chisq_5k = pd.concat([chisq_5k,tfidf_df_13k_connotations.iloc[:, -4:]],axis=1)
chisq_5k.head()

,007,03,10yr,13th,1896,1910,1913,1927,1936,1937,...,zone,zoom,zorak,zu,zulu,zwick,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,35,18,10,4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20,22,2,5
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,28,26,1,10
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,54,35,13,13
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40,46,10,5


In [18]:
# This will get the top 5000 relavant features out of the sample
chi2_selector = SelectKBest(chi2, k=8000)

# This will transform the dataset i.e, it will reduce the dimensions by just considering the relavant features only
X = df_statistical.drop(columns=['OutputSentiment'])
y = df_statistical['OutputSentiment']
X_8000 = chi2_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X.columns[selected_feature_indices]

chisq_8k = X[selected_feature_names]
chisq_8k.head()

chisq_8k = pd.concat([chisq_8k,tfidf_df_13k_connotations.iloc[:, -4:]],axis=1)
chisq_8k.head()

,007,0083,03,101,10yr,12th,13th,15th,16,17,...,zoomed,zorak,zu,zucker,zulu,zwick,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,35,18,10,4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20,22,2,5
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,28,26,1,10
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,54,35,13,13
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40,46,10,5


In [19]:
chisq_8k

,007,0083,03,101,10yr,12th,13th,15th,16,17,...,zoomed,zorak,zu,zucker,zulu,zwick,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,35,18,10,4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20,22,2,5
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,28,26,1,10
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,54,35,13,13
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40,46,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,66,50,11,8
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.114193,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,26,23,9,2
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31,28,9,9
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,6,1,3


## CHI SQAURE CLASSIFICATION

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

# Multinomial Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_scores = cross_val_score(nb_classifier, chisq_8k, y, cv=5)

print("Multinomial Naive Bayes Cross-Validation Scores:")
print(nb_scores)
print("Mean Accuracy:", np.mean(nb_scores))

# k-Nearest Neighbors Classifier
knn_classifier = KNeighborsClassifier()
knn_scores = cross_val_score(knn_classifier, chisq_5k, y, cv=5)

print("\nk-Nearest Neighbors Cross-Validation Scores:")
print(knn_scores)
print("Mean Accuracy:", np.mean(knn_scores))

Multinomial Naive Bayes Cross-Validation Scores:
[0.768      0.797      0.787      0.799      0.78378378]
Mean Accuracy: 0.7869567567567567

k-Nearest Neighbors Cross-Validation Scores:
[0.692      0.684      0.694      0.739      0.68668669]
Mean Accuracy: 0.6991373373373373


In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Load your data
# Assuming X and y are your features and target variables

# Initialize models
svm_model = SVC(kernel='linear')  # Linear SVM
logistic_model = LogisticRegression()

# Initialize KFold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform 5-fold cross-validation for SVM
svm_scores = cross_val_score(svm_model, chisq_5k, y, cv=kfold)

# Perform 5-fold cross-validation for Logistic Regression
logistic_scores = cross_val_score(logistic_model, chisq_8k, y, cv=kfold)

# Display the cross-validation scores
print("SVM Cross-validation scores:", svm_scores)
print("Logistic Regression Cross-validation scores:", logistic_scores)

# Optionally, you can calculate mean and standard deviation of the scores
print("SVM Mean Accuracy:", np.mean(svm_scores))
print("SVM Standard Deviation of Accuracy:", np.std(svm_scores))
print("Logistic Regression Mean Accuracy:", np.mean(logistic_scores))
print("Logistic Regression Standard Deviation of Accuracy:", np.std(logistic_scores))

c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

SVM Cross-validation scores: [0.863      0.865      0.887      0.871      0.85285285]
Logistic Regression Cross-validation scores: [0.85       0.845      0.852      0.849      0.83283283]
SVM Mean Accuracy: 0.8677705705705707
SVM Standard Deviation of Accuracy: 0.011253202898569698
Logistic Regression Mean Accuracy: 0.8457665665665666
Logistic Regression Standard Deviation of Accuracy: 0.006857139861033929


c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
